<a href="https://colab.research.google.com/github/rautshree/AI-class/blob/master/CBOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Resources
A lot of resources are mentioned here: http://mccormickml.com/2016/04/27/word2vec-resources/

As we know, CBOW is learning to predict the word by the context. Or maximize the probability of the target word by looking at the context. And this happens to be a problem for rare words. For example, given the context `yesterday was a really [...] day` CBOW model will tell you that most probably the word is `beautiful` or `nice`. Words like `delightful` will get much less attention of the model, because it is designed to predict the most probable word. This word will be smoothed over a lot of examples with more frequent words.

On the other hand, the skip-gram model is designed to predict the context. Given the word `delightful` it must understand it and tell us that there is a huge probability that the context is `yesterday was really [...] day`, or some other relevant context. With skip-gram the word delightful will not try to compete with the word beautiful but instead, delightful+context pairs will be treated as new observations.

In [0]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import torch.nn.functional as F
import operator

# Continuous Bag of Words model
class CBOW(nn.Module):

    def __init__(self, context_size=2, embedding_size=100, vocab_size=None):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_size)
        self.linear1 = nn.Linear(embedding_size, vocab_size)

    def forward(self, inputs):
        lookup_embeds = self.embeddings(inputs)
        embeds = lookup_embeds.sum(dim=0)
        out = self.linear1(embeds)
        out = F.log_softmax(out)
        return out



def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)



In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMBEDDING_SIZE = 10
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".lower().split()

# How could you do a better pre-processing?
# Maybe a sentence tokenizer?
# Maybe a word lemmatizer?
# Should you take a bigger corpus, Replace this small corpus with a bigger one
# Maybe you should remove stopwords
# Maybe you should just Google?

In [8]:
# Create the  vocabulary
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))

print (data[0])

(['we', 'are', 'to', 'study'], 'about')


In [10]:
loss_func = nn.CrossEntropyLoss()
net = CBOW(CONTEXT_SIZE, embedding_size=EMBEDDING_SIZE, vocab_size=vocab_size)
optimizer = optim.SGD(net.parameters(), lr=0.01)

# The training loop
for epoch in range(100):
    total_loss = 0
    for context, target in data:
        context_var = make_context_vector(context, word_to_ix)
        net.zero_grad()
        ## Enter code to get log_probs from model
        log_probs = net(context_var)
        target = autograd.Variable(torch.LongTensor([word_to_ix[target]]))
        loss = loss_func(log_probs.reshape(1,-1), target)

        loss.backward()
        optimizer.step()
        total_loss += loss.data
    print("Loss for epoch ", epoch, " : ", total_loss)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Loss for epoch  0  :  tensor(251.2954)
Loss for epoch  1  :  tensor(225.9308)
Loss for epoch  2  :  tensor(206.3562)
Loss for epoch  3  :  tensor(190.2513)
Loss for epoch  4  :  tensor(176.5161)
Loss for epoch  5  :  tensor(164.5693)
Loss for epoch  6  :  tensor(154.0367)
Loss for epoch  7  :  tensor(144.6463)
Loss for epoch  8  :  tensor(136.1925)
Loss for epoch  9  :  tensor(128.5194)
Loss for epoch  10  :  tensor(121.5085)
Loss for epoch  11  :  tensor(115.0685)
Loss for epoch  12  :  tensor(109.1283)
Loss for epoch  13  :  tensor(103.6313)
Loss for epoch  14  :  tensor(98.5315)
Loss for epoch  15  :  tensor(93.7913)
Loss for epoch  16  :  tensor(89.3783)
Loss for epoch  17  :  tensor(85.2647)
Loss for epoch  18  :  tensor(81.4252)
Loss for epoch  19  :  tensor(77.8370)
Loss for epoch  20  :  tensor(74.4787)
Loss for epoch  21  :  tensor(71.3309)
Loss for epoch  22  :  tensor(68.3756)
Loss for epoch  23  :  tensor(65.5966)
Loss for epoch  24  :  tensor(62.9792)
Loss for epoch  25  :

In [0]:
# Now let's find embedding for every word
vocab_to_embedding = {}
for word in vocab:
    vocab_to_embedding[word] = net.embeddings.forward(make_context_vector([word], word_to_ix))

In [0]:
def find_k_similar_words(word, k = 5):
    word = word.lower()
    if word not in vocab:
        print ("Not found ", word)
        return []
    a = vocab_to_embedding[word]
    max_sim = -1
    sim_here = {}
    for b in vocab_to_embedding:
        emb = vocab_to_embedding[b]
        sim = torch.dot(a.reshape(-1),emb.reshape(-1))/(a.norm()*emb.norm())
        sim_here[b] = sim.data[0]
    sorted_t = sorted(sim_here.items(), key=operator.itemgetter(1))
    sorted_t.reverse()
    return sorted_t[:k]

In [13]:
find_k_similar_words('program.', 5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  if sys.path[0] == '':


[('program.', tensor(1.)),
 ('by', tensor(0.4817)),
 ('process.', tensor(0.4519)),
 ('idea', tensor(0.4416)),
 ('process', tensor(0.3935))]

### Could you define a Skip Gram model?